### Import and Check Data

In [9]:
import pandas as pd
df = pd.read_csv('../data/cleaned_df')
# df = df.dropna(subset = ['summary' , 'reviewText'])
df.isna().sum()

/Users/moyang/.pyenv/versions/3.7.1/envs/amazon_project/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


asin                0
title               0
brand           19334
Rating              0
reviewerID          0
reviewTime          0
reviewText          0
summary             0
vote          1778221
rank                0
also_buy            0
also_view           0
dtype: int64

In [10]:
data = df[['summary', 'reviewText']]
data.isna().sum()

summary       0
reviewText    0
dtype: int64

### steps:
* lowercase
* numbers
* punctuations
* stopwords
* lemmatizing


In [1]:
from packages import preprocess

ModuleNotFoundError: No module named 'packages'

In [11]:
import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def remove_punc(text):
    for punc in string.punctuation:
        text = text.replace(punc, '')
    return text

def remove_digit(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text  

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word.lower() for word in word_tokenize(text) if not word in stop_words])
    return text

def lemmatizer(text):
    lst = [WordNetLemmatizer().lemmatize(word) for word in word_tokenize(text)]
    return (' '.join(lst))


data = data.applymap(remove_punc)
data = data.applymap(remove_digit)
data = data.applymap(remove_stopwords)
data = data.applymap(lemmatizer)

### Save preprocessed data to pickle

In [16]:
data.to_pickle("./preprocessed.pkl")

### Try LDA

In [16]:
from sklearn.decomposition import LatentDirichletAllocation as lda
from sklearn.feature_extraction.text import TfidfVectorizer

def get_lda_model(data, n_components=2):
    
    vectorizer = TfidfVectorizer().fit(data)
    
    data_vectorized = vectorizer.transform(data)
     
    lda_model = lda(n_components = n_components).fit(data_vectorized)
    return (lda_model, vectorizer)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [17]:
model, vectorizer = get_lda_model(data['summary'])
print_topics(model, vectorizer)

Topic 0:
[('star', 216206.33491629697), ('five', 176737.80546909876), ('one', 50764.31810980759), ('four', 35913.94048318108), ('work', 35678.964483053314), ('it', 22290.887775410592), ('three', 20662.347981813382), ('perfect', 16001.710268856034), ('two', 15146.883391477872), ('like', 13533.481795811238)]
Topic 1:
[('great', 106163.48674735877), ('game', 102804.84824138266), ('good', 58408.25463317011), ('fun', 42353.54729827685), ('love', 38051.499759474515), ('awesome', 32683.573688821623), ('best', 27377.363624394056), ('buy', 21881.531500314322), ('product', 20999.641478891503), ('the', 18785.26833593498)]
